<a href="https://colab.research.google.com/github/plindma/ST-554-Project1/blob/main/ST554Project1Task2_Loring.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **ST 554 Project 1: Task 2**
---
Authored by: Jamie Loring

Collaborators: Peter Lindmark, Evan Whitfield

### **Installing Required Modules**
Please note that the `ucimlrepo` library needs to be installed before importing the requried dataset. Here is the line of code that needs to be run:

`!pip install ucimlrepo`

### **Importing Required Modules**
The code below imports the modules that will be used to read in the data and conduct EDA.

In [ ]:
import ucimlrepo as uci
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### **Reading in the Dataset and Data Information**
The code below reads in the data from the UCI website. In particular, the *IMPORT IN PYTHON* button was used to write this code.

In [ ]:
# fetch dataset
air_quality = uci.fetch_ucirepo(id=360)

# save the data we specifically need
X = air_quality.data.features
X.head() #see the first few rows of the dataframe

,Date,Time,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
0,3/10/2004,18:00:00,2.6,1360,150,11.9,1046,166,1056,113,1692,1268,13.6,48.9,0.7578
1,3/10/2004,19:00:00,2.0,1292,112,9.4,955,103,1174,92,1559,972,13.3,47.7,0.7255
2,3/10/2004,20:00:00,2.2,1402,88,9.0,939,131,1140,114,1555,1074,11.9,54.0,0.7502
3,3/10/2004,21:00:00,2.2,1376,80,9.2,948,172,1092,122,1584,1203,11.0,60.0,0.7867
4,3/10/2004,22:00:00,1.6,1272,51,6.5,836,131,1205,116,1490,1110,11.2,59.6,0.7888


Here is a description of all the variables:

- “True” (or gold standard) measurements of pollutants in the air
    - CO(GT) (CO concentration)
    - NMHC(GT) (Non Metanic HydroCarbon concentration)
    - C6H6(GT) (Benzene concentration)
    - NOx(GT) (NOx concentration)
    - NO2(GT) (NO2 concentration)
- Sensor measurements of pollutants in the air
    - PT08.S1(CO)
    - PT08.S2(NMHC)
    - PT08.S3(NOx)
    - PT08.S4(NO2)
    - PT08.S5(O3)
- Weather related measurements
    - T (temperature)
    - RH (Relative humidity)
    - AH (Absolute humidity)

### **Goals of EDA and Data Cleanup**
Our goal is to investigate relationships between the sensor and weather variables with the Benzene variable. Thus, I will remove the excesss "True" variables from our dataset and do some column renaming to make the dataset more workable. This new object will be called `air_qual_sub`.

In [ ]:
air_qual_sub = X.drop(columns=["CO(GT)", "NMHC(GT)", "NOx(GT)", "NO2(GT)"]) \
                .rename(columns={"C6H6(GT)": "Benzene",
                                 "PT08.S1(CO)": "CO",
                                 "PT08.S2(NMHC)": "NMHC",
                                 "PT08.S3(NOx)": "NOx",
                                 "PT08.S4(NO2)": "NO2",
                                 "PT08.S5(O3)": "O3",
                                 "T": "Temperature",
                                 "RH": "Rel_Humid",
                                 "AH": "Abs_Humid"})

# recode Date and Time as actual date/time variables (not strings), and sort in nested ascending order
air_qual_sub["Date"] = pd.to_datetime(air_qual_sub["Date"], format="%m/%d/%Y")
air_qual_sub["Time"] = pd.to_datetime(air_qual_sub["Time"], format="%H:%M:%S").dt.time
air_qual_sub = air_qual_sub.sort_values(by=["Date", "Time"])
air_qual_sub.head()

,Date,Time,CO,Benzene,NMHC,NOx,NO2,O3,Temperature,Rel_Humid,Abs_Humid
0,2004-03-10,18:00:00,1360,11.9,1046,1056,1692,1268,13.6,48.9,0.7578
1,2004-03-10,19:00:00,1292,9.4,955,1174,1559,972,13.3,47.7,0.7255
2,2004-03-10,20:00:00,1402,9.0,939,1140,1555,1074,11.9,54.0,0.7502
3,2004-03-10,21:00:00,1376,9.2,948,1092,1584,1203,11.0,60.0,0.7867
4,2004-03-10,22:00:00,1272,6.5,836,1205,1490,1110,11.2,59.6,0.7888


### **Begin EDA**

We will first create a two-way contingency table that shows the average Benzene concentration by Date and Time.

In [ ]:
pd.crosstab(
  air_qual_sub.Date,
  air_qual_sub.Time,
  values = air_qual_sub.Benzene,
  aggfunc = ["mean"])

mean                                                        \
Time       00:00:00 01:00:00 02:00:00 03:00:00 04:00:00 05:00:00 06:00:00   
Date                                                                        
2004-03-10      NaN      NaN      NaN      NaN      NaN      NaN      NaN   
2004-03-11      3.6      3.3      2.3      1.7      1.3      1.1      1.6   
2004-03-12      5.9      6.4      4.1      1.9      1.1      1.0      1.8   
2004-03-13      9.6      7.4      5.4      5.4      6.2      2.6      2.9   
2004-03-14     11.0     11.9      8.6      9.7      5.2      3.7      2.5   
...             ...      ...      ...      ...      ...      ...      ...   
2005-03-31      4.8      4.1      2.5      1.7      1.6      2.6      6.1   
2005-04-01      1.5      1.0      1.1      0.9      0.8      1.4      2.0   
2005-04-02      5.8      1.9      1.5      1.4      0.9      1.4      2.0   
2005-04-03      2.5      2.8      2.7      2.8      2.0      1.0      1.3   
2005-04-04      3.5      1.9      1.5      1.1      0.8      1.3      4.4   

                                       ...                             \
Time       07:00:00 08:00:00 09:00:00  ... 14:00:00 15:00:00 16:00:00   
Date                                   ...                              
2004-03-10      NaN      NaN      NaN  ...      NaN      NaN      NaN   
2004-03-11      3.2      8.0      9.5  ...     11.5      8.8      8.3   
2004-03-12      4.4     17.9     22.1  ...     12.8     14.2     12.7   
2004-03-13      5.1     11.8     15.1  ...      8.6      9.2     10.2   
2004-03-14      2.4      4.2      6.4  ...      7.5      8.2     11.9   
...             ...      ...      ...  ...      ...      ...      ...   
2005-03-31     23.6     10.9      6.8  ...      3.1      3.7      3.8   
2005-04-01      6.7      6.3      4.3  ...      3.3      3.6      4.3   
2005-04-02      3.1      2.5      2.7  ...      2.3      2.5      2.9   
2005-04-03      1.7      2.4      4.1  ...      4.2      5.4      6.3   
2005-04-04     17.4     22.4     13.6  ...     11.9      NaN      NaN   

                                                                           
Time       17:00:00 18:00:00 19:00:00 20:00:00 21:00:00 22:00:00 23:00:00  
Date                                                                       
2004-03-10      NaN     11.9      9.4      9.0      9.2      6.5      4.7  
2004-03-11     11.2     20.8     27.4     24.0     12.8      4.7      2.6  
2004-03-12     11.7     19.3     18.2     32.6     20.1     14.3     21.8  
2004-03-13     10.6     15.5     19.6     19.2     18.3     13.1     10.9  
2004-03-14     12.0     12.2     20.6     23.1     14.7      9.0      8.8  
...             ...      ...      ...      ...      ...      ...      ...  
2005-03-31      6.5      7.2      8.3      4.4      2.3      1.8      2.1  
2005-04-01      5.1      5.5      7.4      3.7      1.9      2.1      3.4  
2005-04-02      3.5      4.0      3.5      2.9      1.8      1.8      2.1  
2005-04-03      6.1      4.6     11.1      7.9      6.0      5.8      5.1  
2005-04-04      NaN      NaN      NaN      NaN      NaN      NaN      NaN  

[391 rows x 24 columns]